In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.stat import Correlation

In [2]:
spark = SparkSession.builder.appName("VarunInsClaim").master("local[5]").enableHiveSupport().getOrCreate()

25/06/24 10:36:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/24 10:36:41 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/24 10:36:41 WARN Utils: Serv

In [3]:
insurance = spark.read.csv("insurancedata/insurance_claims.csv",sep=',',header=True, inferSchema=True)

In [4]:
insurance.show(5)

25/06/24 10:36:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+---+-------------+------------+----------+-----------------+---------------------+--------------+-----------+-----------+-----------------------+------------------+---------------+--------------------+-------------+------------+-------------+--------------------+--------------+-----------------+---------------------+--------------+-----------------+--------------------+------------------------+---------------------------+---------------+---------------+---------+-----------------------+------------------+------------+--------------+-------------+---------+----------+---------+--------------+
|months_as_customer|age|policy_number|policy_state|policy_csl|policy_deductable|policy_annual_premium|umbrella_limit|insured_zip|insured_sex|insured_education_level|insured_occupation|insured_hobbies|insured_relationship|capital_gains|capital_loss|incident_date|       incident_type|collision_type|incident_severity|authorities_contacted|incident_state|    incident_city|   inciden

In [5]:
insurance.printSchema()

root
 |-- months_as_customer: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- policy_number: string (nullable = true)
 |-- policy_state: string (nullable = true)
 |-- policy_csl: string (nullable = true)
 |-- policy_deductable: integer (nullable = true)
 |-- policy_annual_premium: double (nullable = true)
 |-- umbrella_limit: integer (nullable = true)
 |-- insured_zip: integer (nullable = true)
 |-- insured_sex: string (nullable = true)
 |-- insured_education_level: string (nullable = true)
 |-- insured_occupation: string (nullable = true)
 |-- insured_hobbies: string (nullable = true)
 |-- insured_relationship: string (nullable = true)
 |-- capital_gains: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- incident_date: string (nullable = true)
 |-- incident_type: string (nullable = true)
 |-- collision_type: string (nullable = true)
 |-- incident_severity: string (nullable = true)
 |-- authorities_contacted: string (nullable = true)
 |-- inci

In [6]:
null_counts = insurance.select([sum(col(c).isNull().cast('int')).alias(c) for c in insurance.columns])
null_counts.show()

+------------------+---+-------------+------------+----------+-----------------+---------------------+--------------+-----------+-----------+-----------------------+------------------+---------------+--------------------+-------------+------------+-------------+-------------+--------------+-----------------+---------------------+--------------+-------------+-----------------+------------------------+---------------------------+---------------+---------------+---------+-----------------------+------------------+------------+--------------+-------------+---------+----------+---------+--------------+
|months_as_customer|age|policy_number|policy_state|policy_csl|policy_deductable|policy_annual_premium|umbrella_limit|insured_zip|insured_sex|insured_education_level|insured_occupation|insured_hobbies|insured_relationship|capital_gains|capital_loss|incident_date|incident_type|collision_type|incident_severity|authorities_contacted|incident_state|incident_city|incident_location|incident_hour_of_

In [7]:
insurance.count()

2300

In [8]:
insurance = insurance.na.drop()

In [9]:
insurance.count()

1025

In [10]:
insurance = insurance.withColumn("incident_date", to_date(insurance.incident_date))

In [11]:
insurance = insurance.withColumn("incident_year",year(insurance.incident_date))

In [12]:
insurance.select(insurance.incident_date).show(5)

+-------------+
|incident_date|
+-------------+
|   2024-01-26|
|   2022-12-08|
|   2022-04-08|
|   2021-02-11|
|   2022-03-18|
+-------------+
only showing top 5 rows



In [13]:
insurance.select(insurance.incident_year).show(5)

+-------------+
|incident_year|
+-------------+
|         2024|
|         2022|
|         2022|
|         2021|
|         2022|
+-------------+
only showing top 5 rows



In [14]:
insurance.write.mode("overwrite").partitionBy("incident_year").parquet("insurancedata/parquet")

In [16]:
spark.sql("select * from varunmdb.insurance_claim limit 10").show()

25/06/24 10:37:16 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+------------------+---+-------------+------------+----------+-----------------+---------------------+--------------+-----------+-----------+-----------------------+------------------+---------------+--------------------+-------------+------------+-------------+--------------------+---------------+-----------------+---------------------+--------------+------------------+--------------------+------------------------+---------------------------+---------------+---------------+---------+-----------------------+------------------+------------+--------------+-------------+---------+----------+---------+--------------+-------------+
|months_as_customer|age|policy_number|policy_state|policy_csl|policy_deductable|policy_annual_premium|umbrella_limit|insured_zip|insured_sex|insured_education_level|insured_occupation|insured_hobbies|insured_relationship|capital_gains|capital_loss|incident_date|       incident_type| collision_type|incident_severity|authorities_contacted|incident_state|     incide

In [21]:
insurance.select(["incident_year", "fraud_reported"]).filter(insurance.fraud_reported=="Y").groupBy(col("incident_year")).count().alias("count").orderBy(col("count").desc()).show()

+-------------+-----+
|incident_year|count|
+-------------+-----+
|         2021|  104|
|         2023|  100|
|         2024|   98|
|         2022|   86|
|         2020|   57|
|         2025|   52|
+-------------+-----+



In [25]:
spark.sql("select incident_year, count(fraud_reported) as fraud_count from varunmdb.insurance_claim where fraud_reported='Y' group by incident_year order by fraud_count desc").show()

+-------------+-----------+
|incident_year|fraud_count|
+-------------+-----------+
|         2021|        104|
|         2023|        100|
|         2024|         98|
|         2022|         86|
|         2020|         57|
|         2025|         52|
+-------------+-----------+



In [42]:
insurance.select(["incident_severity","fraud_reported","total_claim_amount"]).groupBy(col("incident_severity"),col("fraud_reported")).avg("total_claim_amount").show()

+-----------------+--------------+-----------------------+
|incident_severity|fraud_reported|avg(total_claim_amount)|
+-----------------+--------------+-----------------------+
|       Total Loss|             Y|      28088.50311926605|
|   Trivial Damage|             Y|      25490.95492307692|
|     Major Damage|             Y|     25402.379743589736|
|     Minor Damage|             Y|     25120.952907801424|
|     Minor Damage|             N|      27713.14116788321|
|     Major Damage|             N|      24807.60552845529|
|   Trivial Damage|             N|     25434.518214285712|
|       Total Loss|             N|     26098.876484375003|
+-----------------+--------------+-----------------------+



In [45]:
insurance.select(["auto_make", "total_claim_amount"]).orderBy(col("total_claim_amount").desc()).show(5)

+---------+------------------+
|auto_make|total_claim_amount|
+---------+------------------+
|    Honda|          49968.57|
|   Toyota|          49936.21|
|Chevrolet|          49933.36|
|     Ford|          49864.04|
|     Ford|          49821.58|
+---------+------------------+
only showing top 5 rows



In [50]:
fr_cust = insurance.select(['months_as_customer','fraud_reported'])

In [52]:
fr_cust = fr_cust.withColumn("fraud_reported", when(col("fraud_reported")=="Y",1).when(col("fraud_reported")=="N",0))

In [60]:
correlation=fr_cust.stat.corr('months_as_customer','fraud_reported')
correlation

0.010409867906131687

In [61]:
insurance.agg(count('injury_claim')).show()

+-------------------+
|count(injury_claim)|
+-------------------+
|               1025|
+-------------------+



In [62]:
insurance=insurance.withColumn('injury',when(col('injury_claim')>0,1).otherwise(0))\
.withColumn('property',when(col('property_claim')>0,1).otherwise(0))\
.withColumn('vehicle',when(col('vehicle_claim')>0,1).otherwise(0))
claim_type=insurance.groupBy('fraud_reported').agg(sum('injury').alias('injury'),sum('property').alias('property'),sum('vehicle').alias('vehicle'))
claim_type.show()

+--------------+------+--------+-------+
|fraud_reported|injury|property|vehicle|
+--------------+------+--------+-------+
|             Y|   497|     497|    497|
|             N|   528|     528|    528|
+--------------+------+--------+-------+

